In [10]:
import psycopg2
from sqlalchemy import create_engine
from pathlib import Path
import json
DB_CONFIG_PATH = Path(Path.cwd().parent.parent.parent.parent/"settings"/"db_config.json")
with open(str(DB_CONFIG_PATH), "r") as read_file: DB_CONFIG = json.load(read_file)

import pandas as pd
# Replace 'username', 'password', 'host', 'port', and 'database' with your actual database credentials
engine = create_engine('postgresql' + '://{user}:{password}@{host}:{port}/{database}'.format(**DB_CONFIG))


In [154]:
df_report_links = pd.read_sql_query("""
SELECT *
FROM tickers_table 
INNER JOIN 
        report_liks ON report_liks.cik = tickers_table.cik

ORDER BY tickers_table.cik"""
    ,engine) 

df_report_links['date_publication'] = pd.to_datetime(df_report_links['date_publication'],format='%Y-%m-%d')

In [19]:
df_descriptive = pd.read_sql_table(
    table_name='descriptive_info',con=engine)

In [22]:
print(
    df_report_links.columns,
    df_descriptive.columns
)

Index(['id', 'cik', 'ticker', 'title', 'date_added', 'date_publication',
       'quartal', 'url_report'],
      dtype='object') Index(['id', 'symbol', 'longName', 'sector', 'industry', 'country',
       'longBusinessSummary', 'fullTimeEmployees', 'website', 'logo_url',
       'exchange'],
      dtype='object')


In [31]:
grouped_reports = (df_report_links
    .groupby('ticker',as_index=False).agg(
        unique_url = ('url_report',pd.Series.nunique))
    .sort_values(by="unique_url",ascending=False)    
    .rename(columns={"ticker":'symbol'})    )
grouped_reports

,symbol,unique_url
7859,VAL-WT,65
7858,VAL,65
1318,BTCS,64
3210,GE,62
5090,MSI,60
...,...,...
3791,HUDA,1
7919,VHLD,1
7099,SOYB,1
6701,SATL,1


In [138]:
grouped_desc_grouped_reports = (
    df_descriptive.merge(grouped_reports,on='symbol')
)
grouped_desc_grouped_reports.head()

,id,symbol,longName,sector,industry,country,longBusinessSummary,fullTimeEmployees,website,logo_url,exchange,unique_url
0,1,A,"Agilent Technologies, Inc.",Healthcare,Diagnostics & Research,United States,"Agilent Technologies, Inc. provides applicatio...",17200.0,https://www.agilent.com,https://logo.clearbit.com/agilent.com,NYQ,57
1,2,AA,Alcoa Corporation,Basic Materials,Aluminum,United States,"Alcoa Corporation, together with its subsidiar...",12200.0,https://www.alcoa.com,https://logo.clearbit.com/alcoa.com,NYQ,26
2,3,AAC,Ares Acquisition Corporation,Financial Services,Shell Companies,United States,Ares Acquisition Corporation does not have sig...,NaN,https://www.aresacquisitioncorporation.com,https://logo.clearbit.com/aresacquisitioncorpo...,NYQ,10
3,4,AACG,ATA Creativity Global,Consumer Defensive,Education & Training Services,China,"ATA Creativity Global, together with its subsi...",NaN,https://www.atai.net.cn,https://logo.clearbit.com/atai.net.cn,NGM,10
4,5,AACI,Armada Acquisition Corp. I,Financial Services,Shell Companies,United States,Armada Acquisition Corp. I focuses on effectin...,NaN,None,,NGM,8


In [35]:
grouped_desc_grouped_reports['fullTimeEmployees'] = grouped_desc_grouped_reports['fullTimeEmployees'].apply(pd.to_numeric)

In [39]:
df_companies = grouped_desc_grouped_reports[['symbol',"sector",'industry','country',"longName","longBusinessSummary",'fullTimeEmployees','unique_url']]

In [112]:
df_companies.sector.value_counts()

sector
Healthcare                1042
Financial Services         632
Technology                 623
Industrials                511
Consumer Cyclical          448
Real Estate                216
Communication Services     186
Consumer Defensive         182
Energy                     156
Basic Materials            140
Utilities                   80
                             6
Name: count, dtype: int64

In [128]:
worked_df = (
    df_companies
    .query("unique_url >40")
    .groupby(["industry",'sector']).head(5)
)
selected_first_5 = (
    worked_df.query('sector == "Healthcare"')
    .sort_values(by=['industry'],ascending=False))


In [189]:
pd.set_option('display.max_colwidth', 50)
 
test_group = (
    
    pd.merge(
        df_report_links
            [["ticker",'date_publication','quartal','url_report']],
        selected_first_5.rename(columns={"symbol":'ticker'})
            [['sector','industry','ticker']],
    on='ticker'
    )
    .drop_duplicates().sort_values(by=['ticker',"date_publication"])
    # .style.set_properties(**{'text-align': 'center', 'text_wrap': True}) 
)
test_group.ticker.unique()

array(['A', 'ABC', 'ABEO', 'ABIO', 'ABT', 'ABVC', 'ACAD', 'ACER', 'ACHC',
       'ACOR', 'ACRX', 'ADMP', 'ADUS', 'AEMD', 'AHPI', 'ALGN', 'ALIM',
       'ALR', 'AMED', 'AMEH', 'ANGO', 'ANIK', 'ANIP', 'APDN', 'ATR',
       'ATRC', 'ATRI', 'AWH', 'AXDX', 'AZTA', 'BIIB', 'BIMI', 'BMY',
       'CAH', 'CEMI', 'CNC', 'CPSI', 'CVS', 'GILD', 'HSIC', 'HSTM',
       'HZNP', 'JNJ', 'LFMD', 'LFMDP', 'MCK', 'MDRX', 'MDVL', 'NRC',
       'NXGN', 'OMI', 'PETS', 'UNH'], dtype=object)